In [1]:
# Jupyter Notebook Cell 1: Import required libraries
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append(r'C:/Users/HFNV/diamondgui')

import measurement  # This should now work if measurement/__init__.py exists
from plotly import graph_objects as go

In [2]:
def replace_slashes(input_string):
    return input_string.replace('\\', '/')



In [3]:
file_path = r"C:\Users\HFNV\diamondgui\temp\backup\250704h13m49s16_MWPhaseTune-default.pkl"
new_path = replace_slashes(file_path)
print(new_path)


C:/Users/HFNV/diamondgui/temp/backup/250704h13m49s16_MWPhaseTune-default.pkl


In [4]:
data=pickle.load(open(new_path, 'rb'))

C:\Users\HFNV\AppData\Local\Temp\ipykernel_25864\1862109767.py:1: ResourceWarning: unclosed file <_io.BufferedReader name='C:/Users/HFNV/diamondgui/temp/backup/250704h13m49s16_MWPhaseTune-default.pkl'>
  data=pickle.load(open(new_path, 'rb'))


In [5]:
xx=data[1]['dataset']['phasevolt']

In [6]:
sig=data[1]['dataset']['signal']
bg=data[1]['dataset']['background']

In [7]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=xx, y=sig-bg, mode='lines', name='Signal'))
# fig.add_trace(go.Scatter(x=xx, y=bg, mode='lines', name='Background'))
fig.update_layout(xaxis_title='Phase Voltage [V]', yaxis_title='Signal [V]')
fig.show()

In [8]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import curve_fit

# Example sine function model
def sin_cos_model(x, A, omega, phi, C):
    return A * np.sin(omega * x + phi) + C

# Fit the model to your data
initial_guess = [0.0001, 4.2, 0, -0.0005]
popt, pcov = curve_fit(sin_cos_model, xx, sig - bg, p0=initial_guess)

# Extract fitted parameters
A_fit, omega_fit, phi_fit, C_fit = popt
period = 2 * np.pi / omega_fit
print(f"Fitted period: {period:.4f}")

# Generate fitted curve
x_fit = np.linspace(np.min(xx), np.max(xx), 1000)
y_fit = sin_cos_model(x_fit, *popt)

# Compute phase-zero crossings (sin argument = 0 mod 2π)
x0 = (-phi_fit) / omega_fit  # First zero crossing
n_crossings = int((np.max(xx) - x0) / (period / 2)) + 1
zero_crossings = x0 + (np.arange(-5, n_crossings + 5) * (period / 2))
zero_crossings = zero_crossings[(zero_crossings >= np.min(xx)) & (zero_crossings <= np.max(xx))]
# Print phase = 0 crossing points
print("Phase difference = 90 (in x):")
for i, x_cross in enumerate(zero_crossings):
    print(f"  Crossing {i+1}: x = {x_cross:.5f}")
# Compute y-values at zero crossings (should be ~C)
y_zeros = sin_cos_model(zero_crossings, *popt)

# Plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=xx, y=sig - bg, mode='markers', name='Data'))
fig.add_trace(go.Scatter(x=x_fit, y=y_fit, mode='lines', name='Sine Fit'))

# Add zero-crossing markers
fig.add_trace(go.Scatter(
    x=zero_crossings, y=y_zeros,
    mode='markers',
    marker=dict(color='red', size=8, symbol='x'),
    name='Phase difference = 90'
))

fig.update_layout(
    title=f"Sine Fit with Period = {period:.4f}",
    xaxis_title='x',
    yaxis_title='sig - bg',
    legend=dict(x=0.01, y=0.99)
)
fig.show()


Fitted period: 1.4086
Phase difference = 90 (in x):
  Crossing 1: x = 6.31460
  Crossing 2: x = 7.01892
  Crossing 3: x = 7.72324
  Crossing 4: x = 8.42756
